# Mistral 7b v0.3 fine-tuning for smishing detection

based on the tutorial ["Mistral 7B Tutorial: A Step-by-Step Guide to Using and Fine-Tuning Mistral 7B"](https://www.datacamp.com/tutorial/mistral-7b-tutorial)

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl wandb
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
# Login to Hugging Face and Weights and Biases

from huggingface_hub import login
from google.colab import userdata

hf_token = userdata.get('HuggingFace')

login(token = hf_token)

wb_token = userdata.get('wandb')

wandb.login(key=wb_token)
run = wandb.init(
    project='Smishing detection with fine-tuned Mistral 7B v0.3',
    job_type="training",
    anonymous="allow"
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: danielhenel (danielhenel-research). Use `wandb login --relogin` to force relogin


In [ ]:
# Model from Hugging Face hub
base_model = 'mistralai/Mistral-7B-Instruct-v0.3'
# Fine-tuned model
new_model = "./models/smishing-detection-mistral-7b-instruct-v0.3"
# Dataset
dataset = load_dataset("text", data_files="./data/mistral_train_data.txt", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

Making the fine-tunning more efficient by using 4-bit quantization

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

Loading the Llama 2 model and tokenizer

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

(True, True)

Parameter-Efficient Fine-Tuning

In [ ]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)

Training parameters

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

Supervised fine-tuning

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:278: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/l

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
25,2.261000
50,1.209700
75,1.760300
100,1.047700
125,1.808300
150,1.026600
175,1.851500
200,0.983100
225,1.823200
250,1.003600


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in

TrainOutput(global_step=2230, training_loss=1.1713368205211623, metrics={'train_runtime': 2054.8586, 'train_samples_per_second': 4.34, 'train_steps_per_second': 1.085, 'total_flos': 3.0118404398505984e+16, 'train_loss': 1.1713368205211623, 'epoch': 2.0})

In [ ]:
wandb.finish()
model.config.use_cache = True

train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,▄▂▃▆▁▂▂▁▂▃▃▄▂▁▁▁▃▃▃▆▂▂▂▃▅▄▅▆▇▃▇▂▅▅▅▅▄▃▄█
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,█▅▆▆▂▂▁▁▅▅▅▅▁▁▁▁▄▅▅▄▂▂▂▂▁▁▂▁▂▂▂▂▁▁▁▂▂▂▂▁
total_flos,3.0118404398505984e+16
train/epoch,2.0
train/global_step,2230
train/grad_norm,1.71205
train/learning_rate,0.0002
train/loss,0.9208


Save the model and the tokenizer

In [ ]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('/content/drive/MyDrive/1/models/smishing-detection-mistral-7b-instruct-v0.3/tokenizer_config.json',
 '/content/drive/MyDrive/1/models/smishing-detection-mistral-7b-instruct-v0.3/special_tokens_map.json',
 '/content/drive/MyDrive/1/models/smishing-detection-mistral-7b-instruct-v0.3/tokenizer.model',
 '/content/drive/MyDrive/1/models/smishing-detection-mistral-7b-instruct-v0.3/added_tokens.json',
 '/content/drive/MyDrive/1/models/smishing-detection-mistral-7b-instruct-v0.3/tokenizer.json')

In [ ]:
trainer.model.push_to_hub("danielhenel/smishing-detection-mistral-7b-instruct-v0.3")

adapter_model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/danielhenel/smishing-detection-mistral-7b-instruct-v0.3/commit/f73991bc27afee41d633e929810f9d5c4d6fbc4f', commit_message='Upload model', commit_description='', oid='f73991bc27afee41d633e929810f9d5c4d6fbc4f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("danielhenel/smishing-detection-mistral-7b-instruct-v0.3")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/danielhenel/smishing-detection-mistral-7b-instruct-v0.3/commit/6168e69849d1b63d06bbca4364ce656f84fc4136', commit_message='Upload tokenizer', commit_description='', oid='6168e69849d1b63d06bbca4364ce656f84fc4136', pr_url=None, pr_revision=None, pr_num=None)